# Silver層への新規データ統合（MERGE処理）

このノートブックでは、Bronze層の新規データをSilver層に統合します。MERGE INTO文を使用して更新・追加処理を行います。

## 目的
- **増分データ処理**: 新規データのみを効率的に処理
- **データ整合性**: 重複データの適切な処理
- **ACID準拠**: トランザクション保証による信頼性

## MERGE処理の特徴
- **UPSERT操作**: 更新（UPDATE）と挿入（INSERT）の組み合わせ
- **条件分岐**: 既存データの有無による処理分岐
- **パフォーマンス**: 全件処理より高速

## 処理ステップ
1. Bronze層の新規データ読み込み
2. データ品質チェックとフィルタリング
3. Silver層への MERGE INTO 実行


In [0]:
# 環境変数の定義
catalog_name = "users"
schema_name = "yukiteru_koide"
silver_table = "transactions_enriched"

# カタログとスキーマの指定
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

In [0]:

# CATALOGとSCHEMAを切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

# Bronze読み込み
df_bronze = spark.read.table(f"{catalog_name}.{schema_name}.bronze_transactions")

# 不要なカラムを除外してフィルタリング
df_new = (
    df_bronze
    .filter("quantity IS NOT NULL")
    .select(
        "transaction_id",
        "customer_id",
        "product_id",
        "store_id",
        "quantity",
        "transaction_date",
        "total_price"
    )
)

# 一時ビューとして登録
df_new.createOrReplaceTempView("updates")

# MERGE INTO：更新または追加
spark.sql(f"""
MERGE INTO {catalog_name}.{schema_name}.{silver_table} AS target
USING updates AS source
ON target.transaction_id = source.transaction_id
WHEN MATCHED THEN UPDATE SET
  target.customer_id = source.customer_id,
  target.product_id = source.product_id,
  target.store_id = source.store_id,
  target.quantity = source.quantity,
  target.transaction_date = source.transaction_date,
  target.total_price = source.total_price
WHEN NOT MATCHED THEN INSERT (
  transaction_id, customer_id, product_id, store_id, quantity, transaction_date, total_price
) VALUES (
  source.transaction_id, source.customer_id, source.product_id, source.store_id, source.quantity, source.transaction_date, source.total_price
)
""")
